In [1]:
import numpy as np
import os
import re
import time
import random
import copy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
class Player:
    def __init__(self, driver):
        self.driver = driver
        self.grid = self.driver.find_element(By.CLASS_NAME, "tile-container")
        self.page = self.driver.find_element(By.XPATH, '//html')
        self.update()
        
        self.possible_moves = [self.move_left, self.move_right, self.move_up, self.move_down]

    def __str__(self):
        return f"{self.game}"
        
    def update(self):
        self.tiles = self.grid.find_elements(By.XPATH, '*')
        
        tiles_names = []
        for tile in self.tiles:
            tiles_names.append(tile.get_attribute("class"))

        tiles_dict = {}
        for tile in tiles_names:
            info = re.findall(r'\d+', tile)
            tiles_dict[info[1]+info[2]] = int(info[0])

        game = np.zeros((4,4))
        for tile in tiles_dict:
            game[int(tile[1:2])-1][int(tile[0:1])-1] = tiles_dict.get(tile)

        self.game = game
    
    def move_left(self):
        self.page.send_keys(Keys.LEFT)
        time.sleep(0.05)
        self.update()
        # print('Moving Left')
        # print(self.game)
    
    def move_right(self):
        self.page.send_keys(Keys.RIGHT)
        time.sleep(0.05)
        self.update()
        # print('Moving Right')
        # print(self.game)
        
    def move_up(self):
        self.page.send_keys(Keys.UP)
        time.sleep(0.05)
        self.update()
        # print('Moving Up')
        # print(self.game)
        
    def move_down(self):
        self.page.send_keys(Keys.DOWN)
        time.sleep(0.05)
        self.update()
        # print('Moving Down')
        # print(self.game)
    
    def move(self, index):
        self.possible_moves[index]()
    
    def ai_move(self, searches_per_move, search_length):
        scores = np.zeros(4)
        
        for i in range(4):
            first_game = Game(self.game)
            if first_game.is_valid(i):
                first_game.move(i)
                first_game.add_new_tile()
                scores[i] += first_game.score
#                 print("W")
            else:
#                 print("L")
                continue
            
            for j in range(searches_per_move):
                move_number = 1
                search_game = copy.deepcopy(first_game)
                valid = True
                
                while valid and move_number < search_length:
                    index = random.randint(0,3)
                    valid = search_game.is_valid(index)
                    
                    if valid:
                        search_game.move(index)
                        search_game.add_new_tile()
                        scores[i] += search_game.score
        best_move_index = np.argmax(scores)
        self.move(best_move_index)
        # print(scores)
    
    def game_over(self):
        temp = Game(self.game)
        for i in range(4):
            if temp.is_valid(i):
                return False
        return True

In [3]:
class Game:
    def __init__ (self, game):
        self.game = game
        self.score = 0
        
        self.possible_moves = [self.move_left, self.move_right, self.move_up, self.move_down]
        
    def __str__(self):
        return f"{self.game}"
    
    def stack(self):
        new_grid = np.zeros((4,4))
        for i in range(4):
            position = 0
            
            for j in range(4):
                if self.game[i][j] != 0:
                    new_grid[i][position] = self.game[i][j]
                    position += 1
                    
        self.game = new_grid
        
    def combine(self):
        for i in range(4):
            for j in range(3):
                if self.game[i][j] != 0 and self.game[i][j] == self.game[i][j+1]:
                    self.game[i][j] *= 2
                    self.game[i][j+1] = 0
                    self.score += self.game[i][j]
                    
    def add_new_tile(self):
        i = random.randint(0,3)
        j = random.randint(0,3)
        while(self.game[i][j] != 0):
            i = random.randint(0,3)
            j = random.randint(0,3)
        self.game[i][j] = random.choice([2,4])
        
    def move_left(self):
        self.stack()
        self.combine()
        self.stack()
        
    def move_right(self):
        self.game = np.flip(self.game, 1)
        self.stack()
        self.combine()
        self.stack()
        self.game = np.flip(self.game, 1)
        
    def move_up(self):
        self.game = self.game.transpose()
        self.stack()
        self.combine()
        self.stack()
        self.game = self.game.transpose()
        
    def move_down(self):
        self.game = self.game.transpose()
        self.game = np.flip(self.game, 1)
        self.stack()
        self.combine()
        self.stack()
        self.game = np.flip(self.game, 1)
        self.game = self.game.transpose()
        
    def move(self, index):
        self.possible_moves[index]()
   
    def is_valid(self, index):
        temp = copy.deepcopy(self)
        temp.possible_moves[index]()
        if np.array_equal(self.game, temp.game):
            return False
        return True
        

In [4]:
scores = []
count = 0


In [5]:
option = Options()
option.add_argument('--disable-notifications')
driver = webdriver.Chrome(executable_path= "chromedriver.exe", chrome_options= option)
driver.get("https://doctorflu.github.io/")


C:\Users\helen\AppData\Local\Temp\ipykernel_9232\48940843.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= "chromedriver.exe", chrome_options= option)
C:\Users\helen\AppData\Local\Temp\ipykernel_9232\48940843.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path= "chromedriver.exe", chrome_options= option)


In [6]:
p = Player(driver)
score = 0

try_again = driver.find_element(By.CLASS_NAME, "retry-button")
keep_playing = driver.find_element(By.CLASS_NAME, "keep-playing-button")
score_board = driver.find_element(By.CLASS_NAME, "score-container")

In [8]:
while score < 50000:
    while p.game_over() == False:
        score_info = re.findall(r'\d+', score_board.text)
        score = int(score_info[0])
        searches_per_move = int(score/20) + 20
        search_length = int(score/200) + 2
        # print(searches_per_move, search_length)

        p.ai_move(searches_per_move, search_length)
        try:
            keep_playing.click()
        except:
            continue
    
    score_info = re.findall(r'\d+', score_board.text)
    score = int(score_info[0])
    print(score)
    scores.append(score)
    time.sleep(2)
    p.page.screenshot(str(count) + "a.png")
    count += 1
    
    time.sleep(2)

    try:
        keep_playing.click()
    except:
        try_again.click()
        
    time.sleep(2)
    p.update()

16048
6980
7052
7196
16120
3044
14484
16044
13704
6392
7128
16076


TypeError: 'NoneType' object is not iterable

In [10]:
print(scores)

[14316, 35924, 16404, 12016, 25552]
